# Problem 4: File Upload Vulnerabilities and Hash Analysis

### Intro
In this exercise, we explore file upload vulnerabilities using DVWA. The goal is to analyze and bypass upload restrictions at different security levels, inject malicious PHP backdoors, and evaluate defenses like MIME type checks and ModSecurity rules. Additionally, the exercise covers file hash analysis, polyglot file creation with ExifTool, and password hash cracking using rainbow tables and Hashcat.

### Source Code of PHP Backdoor

![Hash passwords](imag/bd_4.png)

The following image shows the content of the malicious `backdoor.php` file created for the upload attack. This file contains a simple PHP script designed to allow remote command execution via a GET parameter called `cmd`.

![Hash passwords](imag/4_1.jpeg)

When trying to upload the `backdoor.php` file with its true MIME type, the server rejected the request. As shown in the image, DVWA (medium mode) checks the MIME type and only accepts files reported as `image/jpeg` or `image/png`. This rejection demonstrates the basic protection in place and sets the stage for the next bypass attempt using MIME spoofing.

![Hash passwords](imag/4_2.jpeg)

After the rejection, the attacker used Burp Suite to spoof the MIME type of the malicious file by changing the `Content-Type` from `application/x-php` to `image/jpeg`. This tricked DVWA into accepting the `.php` file as if it were a valid image. The server responded with a success message, confirming that the file `backdoor.php` was successfully uploaded and stored in the `uploads` directory.

![Hash passwords](imag/4_3.jpeg)

The DVWA interface confirms that the upload was successful. This occurred under the "medium" security setting, where the server accepted the spoofed `image/jpeg` MIME type without verifying the file’s actual contents. 


### File Hashes (MD5 and SHA-256)

To ensure integrity and track the uploaded malicious files, cryptographic hashes were calculated using `md5sum` and `sha256sum` for both files

![Hash passwords](imag/4_4.jpeg)

![Hash passwords](imag/4_5.jpeg)


**backdoor.php**
- MD5: `463d58123b5eaa1666eb3d1275e7fbec`
- SHA-256: `a5b1564f3195c98730e3f1674ec0486b6a21b7511e499fcbae7270b0dca106f07`

This file is a pure PHP script uploaded using MIME spoofing to bypass DVWA’s medium-level upload restrictions. It was designed to allow remote command execution through a `cmd` parameter.

**shell.php.png**
- MD5: `f2b05780da625d811f3cf5fb57809777`
- SHA-256: `1c0b10282f9e5c5db4c3af6075eb9f71215534ce391a4f8e61161de755e2059d`

This is a polyglot file: a `.png` image that contains embedded PHP code inside its Exif metadata using `exiftool`. It was crafted to evade stricter security checks and was successfully uploaded under DVWA’s high security level.

These hashes serve as fingerprints of the uploaded files and are useful for later comparison or forensic analysis.

![Hash passwords](imag/4_6.jpeg)

This image shows the DVWA web interface with a success message indicating that the file `shell.php.png` was successfully uploaded.


### ModSecurity Rules

![Hash passwords](imag/4_7.jpeg)

It serves as a reference for verifying whether the file was successfully blocked by custom rules.

![Hash passwords](imag/4_8.jpeg)

A set of ModSecurity rules were written to detect and block malicious file uploads. These rules include blocking `.php` extensions, double extensions like `.php.png`, suspicious MIME types (application/x-php), PHP code in file contents and even in image metadata (Exif). This configuration directly implements a defense-in-depth strategy by detecting multiple attack vectors used in web shell upload attempts.

![Hash passwords](imag/4_9.jpeg)

This shows the main ModSecurity configuration file (`modsecurity.conf`), where the custom rules file is explicitly included using the directive `Include /etc/modsecurity/custom_rules.conf`. This step ensures that the rules defined in the previous image are actually loaded and enforced by the ModSecurity engine during request inspection, completing the WAF configuration process.

### Rainbow Table Generation Process and Lookup Results

![Hash passwords](imag/4_10.jpeg)

This image shows the creation of a custom wordlist named `custom_wordlist4.txt`, which contains a small set of commonly used passwords such as `admin`, `123456`, and `dragon`. These words will be used to generate corresponding MD5 hashes for the rainbow table.

![Hash passwords](imag/4_11.jpeg)

This is the file `rainbow_table.txt`, which contains the MD5 hashes of each password from the wordlist. This table acts as a lookup database to quickly identify plaintext passwords corresponding to known hashes during cracking.


![Hash passwords](imag/4_12.jpeg)

This shows the actual commands used to generate the rainbow table using a loop that calculates MD5 hashes from the wordlist. Additionally, a hash (`21232f297a57a5a743894a0e4a801fc3`) is written to `hash_to_crack.txt`—this is the hash we aim to crack.


![Hash passwords](imag/4_13.jpeg)

Hashcat initializing with configuration details. It shows that one hash will be tested against the custom wordlist. A warning is also shown due to the use of an unoptimized backend, meaning cracking performance might be reduced.

![Hash passwords](imag/4_14.jpeg)

This image shows Hashcat beginning the cracking process using the `custom_wordlist4.txt` against the hash in `hash_to_crack.txt`. It confirms that the dictionary was loaded and that 10 passwords will be tested.


![Hash passwords](imag/4_15.jpeg)

Hashcat successfully cracked the hash `21232f297a57a5a743894a0e4a801fc3` and identified the original plaintext password as `admin`. The cracking process used the rainbow table created earlier and validated the effectiveness of dictionary-based attacks.


Several techniques were used to bypass DVWA’s file upload protections:

- **MIME Type Spoofing**: With Burp Suite, the `Content-Type` of the request was modified from `application/x-php` to `image/jpeg`, successfully tricking the DVWA server into accepting the PHP file (`backdoor.php`) during the "medium" security level.

- **Double Extension Bypass**: The file `shell.php.png` was created to circumvent extension-based filtering. Although it appears to be an image, it contains executable PHP code either directly or in its metadata.

- **ExifTool Polyglot Payload**: PHP code was inserted into the image’s metadata using ExifTool:
  ```bash
  exiftool -Comment='<?php system($_GET["cmd"]); ?>' image.png
  mv image.png shell.php.png

### Recommended Secure File Upload Practices

When it comes to file uploads, protecting systems from malicious content and unauthorized access is essential—especially in distributed environments such as media production teams collaborating across regions. Improper handling of uploaded files can expose servers to malware, server-side script execution, data breaches, or complete system compromise.

To mitigate such risks, the following best practices are strongly recommended:

1. **Authenticate users**  
   Enforce secure authentication mechanisms before allowing uploads. At minimum, implement two-factor authentication (2FA), and ideally multi-factor authentication (MFA), to verify the identity of uploaders.

2. **Implement access and download controls**  
   Restrict who can upload, view, or download files based on roles and permissions. Use models like RBAC (Role-Based Access Control) or ABAC (Attribute-Based Access Control) to enforce access policies.

3. **Conduct regular security audits**  
   Periodically audit your file upload workflows and server configurations to identify vulnerabilities, misconfigurations, or permission escalations. Include penetration testing and third-party code reviews.

4. **Ensure compliance with data protection regulations**  
   Align file handling workflows with security and privacy regulations such as GDPR, HIPAA, or SOC 2. Ensuring compliance protects users and avoids legal penalties.

5. **Train employees on secure file handling**  
   Security awareness is critical. Educate all team members on the risks of insecure file uploads and ensure they follow protocols like avoiding unsafe links, using strong passwords, and verifying upload sources.

6. **Restrict file types**  
   Only allow specific file types (e.g., `.jpg`, `.png`, `.pdf`) based on application needs. Block potentially dangerous formats such as `.php`, `.exe`, `.bat`, or `.sh`.

7. **Store files in secure cloud environments**  
   Choose cloud storage platforms that offer encryption, access controls, and logging. Avoid storing uploaded files directly in publicly accessible or web-exposed directories.

8. **Encrypt files in transit and at rest**  
   Use strong encryption protocols such as TLS 1.2+ for files in transit and AES-256 for data at rest. This ensures that even if files are intercepted or accessed without authorization, they remain unreadable.

9. **Scan files for malware**  
   Automatically scan uploaded files for viruses, trojans, and other malware using antivirus engines or sandboxed analysis environments.

10. **Keep software up to date**  
    Regularly patch and update web servers, libraries, frameworks, and third-party upload components. Known vulnerabilities in outdated systems are frequent attack targets.

11. **Log all upload activity and enable auditing**  
    Maintain logs of file upload attempts, user access, and file downloads. Audit trails help in tracing malicious behavior or incident response investigations.

12. **Sanitize file names**  
    Remove or replace potentially dangerous characters from uploaded file names. Avoid storing sensitive information in file names, and enforce consistent naming conventions.

---

By applying these practices and supplementing them with a well-configured Web Application Firewall (WAF) such as ModSecurity, organizations can greatly reduce their exposure to file upload vulnerabilities. In production environments—especially those that handle sensitive intellectual property, large media files, or cross-border data flows—upload security should never be an afterthought.

